In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymc as pm
import pytensor.tensor as pt
import exoplanet as xo
from astropy.time import Time


In [ ]:

# Read HJD, MAG, MERR from CSV
df = pd.read_csv('data/lc_obs.csv')
time = df['HJD'].values
mag = df['MAG'].values
merr = df['MERR'].values


In [ ]:

# Preprocess time and magnitudes
t0 = time[0]
t_rel = time - t0
t_ref = np.mean(t_rel)
t_norm = t_rel - t_ref

# Convert magnitudes to relative flux
mag0 = np.median(mag)
flux_rel = 10**(-0.4 * (mag - mag0)) - 1.0
flux_err = 0.4 * np.log(10) * (flux_rel + 1) * merr


In [ ]:

# Build and sample exoplanet model
with pm.Model() as model:
    # Orbital period prior
    logP = pm.Normal('logP', mu=np.log(3.21305751), sigma=0.005)
    period = pm.Deterministic('period', pt.exp(logP))

    # Time of transit
    phase = pm.Uniform('phase', lower=-0.5, upper=0.5)
    tm = pm.Deterministic('tm', t_ref + phase * period)

    # Radius ratio and impact parameter
    r = pm.LogNormal('r', mu=np.log(0.1), sigma=0.5)
    b = xo.distributions.ImpactParameter('b', ror=r)

    # Stellar density and radius
    log_rho_star = pm.Normal('log_rho_star', mu=np.log(1.4), sigma=0.5)
    rho_star = pm.Deterministic('rho_star', pt.exp(log_rho_star))

    log_rstar = pm.Normal('log_rstar', mu=np.log(0.7), sigma=0.5)
    r_star = pm.Deterministic('r_star', pt.exp(log_rstar))

    # Keplerian orbit
    orbit = xo.orbits.KeplerianOrbit(period=period, t0=tm, b=b, rho_star=rho_star, r_star=r_star)

    # Limb darkening
    u = xo.distributions.QuadLimbDark('u', initval=[0.3, 0.2])

    # Baseline
    mean = pm.Normal('mean', mu=0.0, sigma=0.1)

    # Light curve model
    raw_lc = (xo.LimbDarkLightCurve(u[0], u[1])
             .get_light_curve(orbit=orbit, r=r, t=t_norm, texp=np.median(np.diff(t_norm)))
             .sum(axis=-1) + mean)

    # Likelihood
    pm.Normal('obs', mu=raw_lc, sigma=flux_err, observed=flux_rel)

    # Sample
    trace = pm.sample(draws=2000, tune=1000, target_accept=0.9)

# Posterior predictive
with model:
    ppc = pm.sample_posterior_predictive(trace, var_names=['raw_lc'], random_seed=42)

lc_samps = ppc.posterior_predictive['raw_lc']
lc_median = np.median(lc_samps, axis=(0,1))
lc_lo, lc_hi = np.percentile(lc_samps, [16,84], axis=(0,1))

# Plot synthetic light curve vs data
plt.figure(figsize=(8,4))
plt.errorbar(time, flux_rel + 1, yerr=flux_err, fmt='.k', label='Data')
plt.plot(time, lc_median + 1, label='Model')
plt.fill_between(time, lc_lo + 1, lc_hi + 1, color='C0', alpha=0.3, label='1σ band')
plt.xlabel('Heliocentric JD')
plt.ylabel('Relative Flux')
plt.legend()
plt.show()